# Interpretación de Modelos de Aprendizaje Automático

El aprendizaje automático impacta cada vez más en decisiones cruciales de salud, seguridad, economía y otras áreas.
Comprender cómo funcionan los modelos de aprendizaje automático y explicar su razonamiento es fundamental.

A mayor interpretabilidad, mayor confianza y transparencia en las decisiones basadas en modelos de aprendizaje automático.

Dificultad de la explicación:
- La diversidad de algoritmos y la naturaleza del aprendizaje automático dificultan la explicación de los modelos.

Solución:
- La interpretabilidad de modelos ayuda a explicar las predicciones de los modelos.

#### Importancia de las Características en los Modelos de Aprendizaje Automático

Comprender la importancia de las características ayuda a interpretar cómo los modelos de aprendizaje automático hacen predicciones para casos individuales y para el conjunto de datos en general.

**Explicando Modelos**:
- Los "explainers" usan técnicas estadísticas para calcular la importancia de las características en un modelo de aprendizaje automático.
- Esto permite cuantificar la influencia relativa de cada característica en la predicción de etiquetas.

**Importancia Global**:
- Mide la importancia general de cada característica en todo el conjunto de datos de prueba.
- Proporciona una comparación general de cómo cada característica influencia la predicción.

Ejemplo:

    Un modelo predice el riesgo de impago de préstamos basado en ingresos, monto del préstamo, edad y estado civil.
    La importancia global podría ser: ingresos (0.98), monto del préstamo (0.67), edad (0.54) y estado civil (0.32).

**Importancia Local**:
- Mide la influencia de cada valor de característica para una predicción individual.
- Se puede usar para entender qué factores influyen en la predicción para una instancia específica.

Ejemplo:

    Se aprueba el préstamo de Sam.
    El explainer indica que el monto del préstamo y luego el ingreso son los más importantes para aprobar el préstamo de Sam.
    Esto difiere de la importancia global, ya que Sam podría tener un ingreso inferior al promedio pero un monto de préstamo inusualmente bajo.


**Modelos Multi-Clase**:
- La importancia local se calcula para cada clase posible.
- La suma de las importancias locales para todas las clases de una característica siempre es cero.

Ejemplo:

    Un modelo predice la especie de pingüino basado en el largo del pico, ancho del pico, largo de la aleta y peso.
    La importancia local de la longitud de la aleta para un pingüino específico podría ser: 0.5 (clase 0), 0.3 (clase 1) y -0.8 (clase 2).

**Modelos de Regresión**:
- No tienen clases, por lo que la importancia local indica la influencia de cada característica en la etiqueta o escala predicha.

#### Uso de Explainers

Se necesita el paquete `azureml-interpret` para crear explicadores.

##### Tipos de Explicadores
- **MimicExplainer**: Crea un modelo sustituto global que aproxima al modelo original y permite generar explicaciones.
  - Requiere la misma arquitectura que el modelo original (ej. lineal, basado en árboles).
- **TabularExplainer**: Envuelve diferentes algoritmos de explicación y elige automáticamente el más adecuado para la arquitectura del modelo.
- **PFIExplainer**: Analiza la importancia de las características barajando sus valores y midiendo el impacto en la predicción.

##### Obtención de Importancia de Características

- **Global**:
  - Se usa `explained_global` y `get_feature_importance_dict` del explicador para obtener un diccionario con la importancia global de cada característica.
  - El código es igual para **MimicExplainer** y **TabularExplainer**.
  - **PFIExplainer** requiere las etiquetas reales de las características de prueba.
  
- **Local**:
  - Se usa `explained_local` del explicador para especificar los casos a explicar.
  - Se utilizan `get_ranked_local_names` y `get_ranked_local_values` para obtener nombres y valores de importancia ordenados.
  - El código es igual para **MimicExplainer** y **TabularExplainer**.
  - **PFIExplainer** no soporta explicaciones de importancia local

#### Creación de explains

Cuando se utiliza un estimador o un script para entrenar un modelo en un experimento de Azure ML, es posible crear un explicador y cargar la explicación generada durante la ejecución para posteriormente analizarlo.

##### 1.Creación de una explicación en el script del experimento

Para crear una explicación en el script del experimento, asegúrese de que los paquetes `azureml-interpret` y `azureml-contrib-interpret` estén instalados en el entorno de ejecución. A continuación, puede utilizarlos para generar una explicación a partir del modelo entrenado y cargarla en las salidas de la ejecución.

In [ ]:
# Generar y cargar una explicación del modelo en un script de experimento

from azureml.core.run import Run
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

run = Run.get_context()

# Codigo de entrenamiento del modelo

# crear un explicador
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# Get an Explanation Client and upload the explanation
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')             # usamos

run.complete()

##### 2.Visualización de la explicación

Para poder ver la explicaciones que se han creado para el modelo en la pestaña **Explicaciones** de la ejecución en Azure ML Studio.

También puede usar el objeto `ExplanationClient` para descargar la explicación en Python.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

client = ExplanationClient.from_run_id(workspace=ws,
                                       experiment_name=experiment.experiment_name, 
                                       run_id=run.id)
explanation = client.download_model_explanation()
feature_importances = explanation.get_feature_importance_dict()

#### Visualizaciones de Importancia de Características en Azure Machine Learning Studio

Las visualizaciones ayudan a explorar la *influencia relativa* de cada característica en las predicciones del modelo.

- Visualizaciones:
  - Importancia global:
    - Muestra la importancia general de cada característica en todo el conjunto de datos de prueba.
    - Permite filtrar para ver solo las K características más importantes usando un control deslizante.

- Distribución de importancia:
  - Muestra la distribución de los valores de importancia individual para cada característica.
  - Ofrece diferentes visualizaciones: diagrama de dispersión (swarm plot), diagrama de cajas (box plot) y diagrama de violín (violin plot).
  - Seleccionar un punto de datos permite ver la importancia local para esa instancia específica.

Extra: [Model interpretability](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-machine-learning-interpretability?view=azureml-api-2)